# Symmetric Matrix Vector Multiply Routines (stored in upper triangle)

This notebook walks you through how to implement $ y := A x + y $ where $ A $ is symmetric with only the upper triangular part stored.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with `include` and `using` statements.

## Algorithm that takes dot products

<img src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/4331_Symv_u_unb_var1.png" alt="Alternative matrix-vector multiplication via dot products algorithm" width="50%">

## The `Symv_u_unb_var1!( A, x, y )` routine

This routine, given symmetric matrix $ A \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := A x + y $.  The "_u_" in the name of the routine indicates that $ A $ is stored in the upper triangular part of the matrix.

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots!( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [14]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Symv_u_unb_var1!(A, x, y)

    ATL, ATR, 
    ABL, ABR  = flame.part_2x2(A, 
                               0, 0, "TL")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "TOP")

    yT, 
    yB  = flame.part_2x1(y, 
                         0, "TOP")

    while size(ATL, 1) < size(A, 1)

        A00,  a01,     A02,  
        a10t, alpha11, a12t, 
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, 
                                                       ABL, ABR, 
                                                       1, 1, "BR")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "BOTTOM")

        y0,   
        psi1, 
        y2    = flame.repart_2x1_to_3x1(yT, 
                                        yB, 
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.dots!( a01, x0, psi1 )
        laff.dots!( alpha11, chi1, psi1 )
        laff.dots!( a12t, x2, psi1 )
        
        #------------------------------------------------------------#

        ATL, ATR, 
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  
                                               a10t, alpha11, a12t, 
                                               A20,  a21,     A22,  
                                               "TL")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "TOP")

        yT, 
        yB  = flame.cont_with_3x1_to_2x1(y0,   
                                         psi1, 
                                         y2,   
                                         "TOP")

    end

    flame.merge_2x1!(yT, 
                     yB, y)

end

Symv_u_unb_var1! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [15]:
A = rand(4, 4)
x = rand(4)
y = rand(4)
yold = rand(4)

println("A before =")
A

A before =


4×4 Array{Float64,2}:
 0.781922   0.0508645  0.309458  0.354825 
 0.541723   0.919095   0.628027  0.389902 
 0.0519269  0.674961   0.150666  0.0903909
 0.400907   0.962833   0.932462  0.240821 

In [16]:
println("x before =")
x

x before =


4-element Array{Float64,1}:
 0.5385822017161468
 0.6353026381807778
 0.3704493491986802
 0.3163403202108439

In [17]:
println("y before =")
y

y before =


4-element Array{Float64,1}:
 0.9896578517413579 
 0.42382469153175895
 0.8587773264237806 
 0.31374423358705017

In [18]:
using LinearAlgebra

# Notice that A is not symmetric.  We now "symmetrize it"
Asymm = Symmetric(A)

laff.copy!( y, yold )   # save the original vector y

Symv_u_unb_var1!( A, x, y )

println( "y after =" )
y

y after =


4-element Array{Float64,1}:
 1.6699853255925794
 1.3911169641210386
 1.5088415779564948
 0.862218689686724 

In [19]:
println( "y - ( Asymm * x + yold )" )
y - ( Asymm * x + yold )

y - ( Asymm * x + yold )


4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

<img
src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/4331_Symv_u_unb_var2.png" alt="Alternative matrix-vector multiplication via dot products algorithm" width="50%">

## The `Symv_u_unb_var2!( A, x, y )` routine

This routine, given symmetric matrix $ A \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := A x + y $.  The "_u_" in the name of the routine indicates that $ A $ is stored in the upper triangular part of the matrix.

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy!( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [20]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Symv_u_unb_var2!(A, x, y)

    ATL, ATR, 
    ABL, ABR  = flame.part_2x2(A, 
                               0, 0, "TL")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "TOP")

    yT, 
    yB  = flame.part_2x1(y, 
                         0, "TOP")

    while size(ATL, 1) < size(A, 1)

        A00,  a01,     A02,  
        a10t, alpha11, a12t, 
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, 
                                                       ABL, ABR, 
                                                       1, 1, "BR")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "BOTTOM")

        y0,   
        psi1, 
        y2    = flame.repart_2x1_to_3x1(yT, 
                                        yB, 
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.axpy!( chi1, a01, y0 )
        laff.axpy!( chi1, alpha11, psi1 )
        laff.axpy!( chi1, a12t, y2 )
        
        #------------------------------------------------------------#

        ATL, ATR, 
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  
                                               a10t, alpha11, a12t, 
                                               A20,  a21,     A22,  
                                               "TL")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "TOP")

        yT, 
        yB  = flame.cont_with_3x1_to_2x1(y0,   
                                         psi1, 
                                         y2,   
                                         "TOP")

    end

    flame.merge_2x1!(yT, 
                     yB, y)

end

Symv_u_unb_var2! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [21]:
A = rand(4, 4)
x = rand(4)
y = rand(4)
yold = rand(4)

println("A before =")
A

A before =


4×4 Array{Float64,2}:
 0.317611  0.0186452  0.0371996  0.163157
 0.771867  0.271247   0.199028   0.670403
 0.523232  0.591103   0.257476   0.982948
 0.56034   0.0518334  0.525471   0.851051

In [22]:
println("x before =")
x

x before =


4-element Array{Float64,1}:
 0.4676976532441679
 0.9893688316355775
 0.8712958868506524
 0.9182635729633037

In [23]:
println("y before =")
y

y before =


4-element Array{Float64,1}:
 0.22930027253768182
 0.8872708334796877 
 0.19433485575566256
 0.03400756051420806

In [24]:
using LinearAlgebra

# Notice that A is not symmetric.  We now "symmetrize it"
Asymm = Symmetric(A)

laff.copy!( y, yold )   # save the original vector y

Symv_u_unb_var2!( A, x, y )

println( "y after =" )
y

y after =


4-element Array{Float64,1}:
 0.5785261649459474
 1.9533727773243834
 1.535588483236333 
 2.411518762201736 

In [25]:
println( "y - ( Asymm * x + yold )" )
y - ( Asymm * x + yold )

y - ( Asymm * x + yold )


4-element Array{Float64,1}:
  1.1102230246251565e-16
 -2.220446049250313e-16 
  0.0                   
 -4.440892098500626e-16 

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.